# Caesar Encryption

In [1]:
def txt_shift(txt, shift):
    result = ""
    for idx in range(0, len(txt)):
        char = txt[idx]
        if char.isalpha():
            if char.isupper():
                order = ord(char) -65 + shift
                order %= 26
                order += 65
            elif char.islower():
                order = ord(char) -97 + shift
                order %= 26
                order += 97
            char = chr(order)
        result += char
    return result

def caesar_encryption(txt, shift):
    return txt_shift(txt, shift)

def caesar_decryption(txt, shift):
    return txt_shift(txt, -1 * shift)

plain_txt = "Hello!"
shift_amount = 10

print(f"原始明文: {plain_txt}")
cipher_txt = caesar_encryption(plain_txt, shift_amount)
print(f"加密密文: {cipher_txt}")
decryption_cipher_txt = caesar_decryption(cipher_txt, shift_amount)
print(f"解密結果: {decryption_cipher_txt}")

原始明文: Hello!
加密密文: Rovvy!
解密結果: Hello!


# Monoalphabetic Encryption

In [2]:
from random import sample

def txt_shift(txt, shift):
    result = ""
    for idx in range(0, len(txt)):
        char = txt[idx]
        if char.isalpha():
            order = ord(char)
            if char.isupper():
                order -= 65
                order = shift[order]
                order += 65
            elif char.islower():
                order -= 97
                order = shift[order]
                order += 97
            char = chr(order)
        result += char
    return result

def mono_encryption(txt, shift):
    return txt_shift(txt, shift)

def mono_decryption(txt, shift):
    inverse_shift = [0] * 26
    for idx, value in enumerate(shift):
        inverse_shift[value] = idx
    return txt_shift(txt, inverse_shift)

plain_txt = "Hello!"
shift_list = sample(range(0,26), 26)

print(f"原始明文: {plain_txt}")
print(f"Monoalphabet: {shift_list}")
cipher_txt = mono_encryption(plain_txt, shift_list)
print(f"加密密文: {cipher_txt}")
decryption_cipher_txt = mono_decryption(cipher_txt, shift_list)
print(f"解密結果: {decryption_cipher_txt}")

原始明文: Hello!
Monoalphabet: [4, 16, 2, 24, 1, 19, 8, 0, 13, 14, 12, 23, 5, 10, 18, 20, 3, 9, 22, 25, 15, 17, 21, 6, 7, 11]
加密密文: Abxxs!
解密結果: Hello!


# XOR Encryption

In [1]:
def string_to_bytes(input):
    input = bytearray(input, 'utf-8')
    result = ""
    for byte in input:
        for i in range(7, -1, -1):
            ## (byte >> i), byte 右移 i bits, ex: '10000' 右移 2 bits 變成 '100'
            ## '&', 二進制 and 運算, ex: 6 & 7, '111' & '110', 運算得到 '110', 也就是 6
            ## 補充: '|', 二進制 or 運算
            result += str((byte >> i) & 1)
    return result

def bytes_to_string(input):
    result = ""
    for idx in range(0, int(len(input)/8)):
        binary = input[8*idx:8*(idx+1)]
        result += chr(int(binary, 2))
    return result

In [37]:
import random

def generate_key(length):
    key = ""
    for i in range(0, length):
        key += str(random.randint(0, 1))
    return key

def xor_operation(text, key):
    if text == key:
        return "0"
    else:
        return "1"

def xor_en_decrypt(text, key):
    result = ""
    len_txt = len(text)
    len_key = len(key)
    for idx in range(0, len_txt):
        if idx >= len_key:
            key_idx = idx % len_key
        else:
            key_idx = idx
        xor_result = xor_operation(text[idx], key[key_idx])
        result += xor_result
    return result

In [38]:
if __name__ == "__main__":
    message = "XOR Cipher!"
    print(f"Origin message: {message}")
    message = string_to_bytes(message)
    print(f"Message in binary: {message}")

    key = generate_key(len(message))
    print(f"Key: {key}")

    encryption = xor_en_decrypt(message, key)
    print(f"Encryption: {encryption}")

    decryption = xor_en_decrypt(encryption, key)
    print(f"Decryption: {decryption}")

    text = bytes_to_string(decryption)
    print(f"Text: {text}")

Origin message: XOR Cipher!
Message in binary: 0101100001001111010100100010000001000011011010010111000001101000011001010111001000100001
Key: 0100011101011001101010111011001001100101001011111011101000001100001111100000011000101001
Encryption: 0001111100010110111110011001001000100110010001101100101001100100010110110111010000001000
Decryption: 0101100001001111010100100010000001000011011010010111000001101000011001010111001000100001
Text: XOR Cipher!


# Substitution-Permutation Network(SPN)

In [1]:
def generate_key(key, rounds):
    key += key
    keys = []
    for idx in range(rounds):
        key_this_round = key[4*idx+4:4*idx+20]
        keys.append(key_this_round)
    return keys

def xor_operation(text, key):
    if text == key:
        return "0"
    else:
        return "1"

def xor_en_decrypt(text, key):
    result = ""
    len_txt = len(text)
    len_key = len(key)
    for idx in range(0, len_txt):
        if idx >= len_key:
            key_idx = idx % len_key
        else:
            key_idx = idx
        xor_result = xor_operation(text[idx], key[key_idx])
        result += xor_result
    return result

In [2]:
def substitution(input, s_box):
    output = ""
    for idx in range(4):
        data = input[4*idx:4*(idx+1)]
        number = int(data, 2)
        number_substitution = s_box[number]
        # Convert to binary string
        # ex: 9 -> "1001"
        binary_number = ""
        for i in range(3, -1, -1):
            binary_number += str((number_substitution >> i) & 1)

        output += binary_number
    return output

In [3]:
def permutation(input, p_box):
    output = list("0" * 16)
    for idx, value in enumerate(p_box):
        output[value] = input[idx]
    return "".join(output)

In [4]:
def spn_encrypt(text, rounds, key, s_box, p_box):
    output = text
    for idx in range(rounds):
        output = xor_en_decrypt(output, key[idx])
        output = substitution(output, s_box)
        output = permutation(output, p_box)
    output = xor_en_decrypt(output, key[rounds])
    return output

In [5]:
def spn_decrypt(text, rounds, key, s_box, p_box):
    output = text
    s_box_inverse = [0]*16
    p_box_inverse = [0]*16
    for idx in range(16):
        s_box_inverse[s_box[idx]] = idx
        p_box_inverse[p_box[idx]] = idx
    for idx in range(rounds):
        output = xor_en_decrypt(output, key[rounds-idx])
        output = permutation(output, p_box_inverse)
        output = substitution(output, s_box_inverse)
    output = xor_en_decrypt(output, key[0])
    return output

In [10]:
import random

if __name__ == '__main__':
    rounds = 3
    key = "1011101000111110"
    keys = generate_key(key, rounds + 1)
    print(f"初始金鑰： {key}")
    print(f"產生金鑰： {keys}")

    s_box = random.sample(range(0, 16), 16)
    print(f"s_box： {s_box}")
    p_box = random.sample(range(0, 16), 16)
    print(f"p_box： {p_box}")

    message = "1001001110100101"
    print(f"原始明文： {message}")
    encryption = spn_encrypt(message, rounds, keys, s_box, p_box)
    print(f"加密密文： {encryption}")
    decryption = spn_decrypt(encryption, rounds, keys, s_box, p_box)
    print(f"原始明文： {decryption}")

初始金鑰： 1011101000111110
產生金鑰： ['1010001111101011', '0011111010111010', '1110101110100011', '1011101000111110']
s_box： [13, 1, 11, 9, 0, 4, 3, 7, 14, 2, 10, 6, 5, 12, 8, 15]
p_box： [9, 13, 4, 8, 0, 2, 11, 12, 3, 7, 1, 10, 15, 5, 14, 6]
原始明文： 1001001110100101
加密密文： 0100000111110101
原始明文： 1001001110100101
